# マイクロマウスのターン生成

## 概要
マイクロマウス競技において、高速な走行を行うためにはスラロームと呼ばれる車体を止めずにターンをする動作が必要とされます。
このプログラムでは、スラローム走行に必要な軌跡を生成します。

## インプット
軌跡を作る際のインプットは以下になります。
    ターンの始点、終点、ターン角度、初期角度、曲線長、角速度変化調整用変数

以下のように直線と曲線を組み合わせてターンを生成します。

    直線 -> 曲線 -> 直線

曲線前後の直線の長さは曲線の長さに応じて自動で決定します。    

## アウトプット
スラロームの軌跡は弧長パラメータ付けされた曲率のリストで出力されます。
ロボットに実装する際には曲率のリストをロボットのプログラムにコピペし、軌道計画に利用してください。
軌道計画の方法についてはREADME.ipynbが参考になるかもしれません。

## マイクロマウス競技で用いるターン

### スラローム レベル 1 
まず、最低限のスラロームとして以下を実装しましょう。

 - 探索90度ターン
 - 最短90度ターン

### スラローム レベル 2
最短走行の際に半径の大きいターンを行うことでより高速で走行が可能です。

 - 大周り90度ターン
 - 180度ターン

### スラローム レベル 3
最短走行の際に区画を斜めに進むことで大幅な走行時間短縮が可能です。
ただし、壁にぶつからないように制御を行うことが少々難しいです。

 - 直進から斜め45度ターン
 - 斜めから直進45度ターン
 - 直進から斜め135度ターン
 - 斜めから直進135度ターン
 - 斜め90度ターン

このうち、45度ターンと135度ターンは直進から斜めと斜めから直進のパターンがありますが、軌跡を逆からたどれば一方が他方になるので、実質3つのターンになります。
### スラローム レベル 4
上記以外にも変則的なスラローム起動が存在します。
(こじまターン等と呼ばれるらしいです。)
    

In [61]:
%matplotlib inline
import matplotlib.pyplot as plt 
import matplotlib.ticker as tick
import numpy as np 
from numba import jit
import math 
from itertools import product

from ipywidgets import interact, interactive, fixed




In [62]:
# molifireの積分値を計算するための関数


#y = math.e / a * np.exp( (-1)/(1- np.abs(2*x/b -1)**c))

@jit
def moli(x,a,b,c):
    return math.e / a * np.exp( (-1)/(1- np.abs(2*x/b -1)**c))

@jit
def int_moli(c):
    delta_t = 0.00001
    x = np.arange(0.000001,1,delta_t)

    S = 0
    y = moli(x,1,1,c)
    for i,ele in enumerate(y):
        if i != 0:
            S = S + delta_t * ele
    return S    


In [102]:

# 軌跡を作るためのクラス
class CalcTraj:
    shape_factor = 0
    path_length = 0
    target_ang = 0
    delta_t = 0
    tread = 0
    start_ang = 0
    start_x = 0
    start_y = 0
    
    s = None
    kappa =  None
    D_kappa = None
    I_kappa = None
    c_traj_x = None
    c_traj_y = None
    l_traj_x = None
    l_traj_y = None
    r_traj_x = None
    r_traj_y = None
    
    # 軌跡のパラメータをセットする
    def set_param(self, shape_factor_, path_length_, target_ang_, delta_t_, tread_, start_x_=0, start_y_=0, start_ang_=0):
        self.shape_factor = shape_factor_
        self.path_length = path_length_
        self.target_ang = target_ang_
        self.delta_t = delta_t_
        self.tread = tread_
        self.start_x = start_x_
        self.start_y = start_y_
        self.start_ang = start_ang_
        
    # 軌跡を計算する    
    @jit
    def calc_traj(self):
        self.s = np.arange(0.00001, self.path_length, self.delta_t)
        
        # a * b * S = target_ang
        c = self.shape_factor
        b = self.path_length
        S = int_moli(c)
        a = 1/(self.target_ang/b/S)
        
        # 曲率 
        self.kappa = np.zeros(len(self.s))
        for i in range(len(self.s)):
            val = 1 - np.abs(2*self.s[i]/b -1)**c
            if val != 0.0:
                self.kappa[i] = math.e / a * np.exp(-1/val) 
            else:
                self.kappa[i] = 0.0
        
        # 曲率の微分
        self.D_kappa = np.gradient(self.kappa)/ self.delta_t

        
        # 曲率の積分
        self.I_kappa = np.zeros(len(self.kappa))
        self.I_kappa[0] = self.start_ang;
        for i,ele in enumerate(self.kappa):
            if i != 0:
                self.I_kappa[i] = self.I_kappa[i-1] + self.delta_t * ele 

        # 軌跡のx座標, y座標を求める        
        self.c_traj_x = np.zeros(len(self.kappa))
        self.c_traj_y = np.zeros(len(self.kappa))

        self.l_traj_x = np.zeros(len(self.kappa))
        self.l_traj_y = np.zeros(len(self.kappa))

        self.r_traj_x = np.zeros(len(self.kappa))
        self.r_traj_y = np.zeros(len(self.kappa))

        self.c_traj_x[0] = self.start_x
        self.c_traj_y[0] = self.start_y
        
        self.r_traj_x[0] = self.c_traj_x[0] + math.sin(self.start_ang)*self.tread/2
        self.r_traj_y[0] = self.c_traj_y[0] - math.cos(self.start_ang)*self.tread/2

        self.l_traj_x[0] = self.c_traj_x[0] - math.sin(self.start_ang)*self.tread/2
        self.l_traj_y[0] = self.c_traj_y[0] + math.cos(self.start_ang)*self.tread/2
        
        for i,ele in enumerate(self.I_kappa):
            if i != 0:
                self.c_traj_x[i] = self.c_traj_x[i-1] + self.delta_t *  math.cos(ele)
                self.c_traj_y[i] = self.c_traj_y[i-1] + self.delta_t *  math.sin(ele)
                self.r_traj_x[i] = self.c_traj_x[i] + math.sin(ele)*self.tread/2
                self.r_traj_y[i] = self.c_traj_y[i] - math.cos(ele)*self.tread/2

                self.l_traj_x[i] = self.c_traj_x[i] - math.sin(ele)*self.tread/2
                self.l_traj_y[i] = self.c_traj_y[i] + math.cos(ele)*self.tread/2
        
            
    # 軌跡が終わったポイントのx座標
    def get_end_x(self):
        return self.c_traj_x[len(self.c_traj_x)-1]
    # 軌跡が終わったポイントのy座標    
    def get_end_y(self):
        return self.c_traj_y[len(self.c_traj_y)-1]
    
    # x方向の移動距離
    def get_move_x(self):
        return self.c_traj_x[len(self.c_traj_x)-1] - self.start_x
    
    # y方向の移動距離
    def get_move_y(self):
        return self.c_traj_y[len(self.c_traj_y)-1] - self.start_y
        
    # 速度一定で走ったときの時間と角速度のリストを返す
    def get_omega(self, v):
        return self.s /v, self.kappa * v
    
    # 速度一定で走ったときの時間と角速度(degree)のリストを返す
    def get_omega_degree(self, v):
        t, rad = self.get_omega(v) 
        return t, rad * 180 /math.pi
    
    # 速度一定で走ったときの時間と角加速度のリストを返す
    def get_alpha(self, v):
        t, rad = self.get_omega(v)
        alpha = np.gradient(rad)/ (self.delta_t/v)
        return t,alpha 

    # 速度一定で走ったときの時間と角加速度(degree)のリストを返す
    def get_alpha_degree(self, v):
        t, alpha = self.get_alpha(v)
        return t, alpha * 180/math.pi
    
    # 速度一定で走ったときの時間と向進加速度のリストを返す
    def get_cent_acc(self, v):
        return self.s/v, v * v * self.kappa 
        
    
    
    # 速度一定で走ったときに右タイヤにかかる力を返す
    def get_r_force(self, v, m, I):
        t, rad = self.get_omega(v)
        t, alpha = self.get_alpha(v)
        omega_f = m * v * 0.5 * rad
        alpha_f = I * alpha / (2 * self.tread)
        f = np.sqrt(omega_f * omega_f + alpha_f * alpha_f)
        return t, omega_f, alpha_f, f
    
    # 速度一定で走ったときに左タイヤにかかる力を返す
    def get_l_force(self, v, m, I):
        t, rad = self.get_omega(v)
        t, alpha = self.get_alpha(v)
        omega_f = m * v * 0.5 * rad
        alpha_f = I * alpha / (2 * self.tread)
        f =  np.sqrt(omega_f * omega_f + alpha_f * alpha_f)
        return t, omega_f, alpha_f, f
        

## クラシック探索90度ターン
壁を読む関係上ターンの開始、終了を1cmずらしています。

In [64]:
from pylab import rcParams
rcParams['figure.figsize'] = 7,7

# 設定パラメータ
tread = 0.03
delta_t = 0.00001
m = 0.02
I = 0.00003
start_x = -0.01 -0.09
start_y = 0.00
end_x = 0.0
end_y = -0.01 + 0.09
target_ang = 90
start_ang = 0.0

shape_factor__ = 2.6
path_length__ = 0.09
v__ = 1.0
traj_cl_serch_90 = CalcTraj()

@interact(shape_factor=(0.1, 8.0, 0.1), path_length=(0.01, 0.36, 0.01),  v = (0.01, 5.0, 0.01)  )
def h(shape_factor=shape_factor__, path_length=path_length__, v=v__):
    traj_curve = traj_cl_serch_90
    traj_curve.set_param(shape_factor_=shape_factor, path_length_= path_length, target_ang_= target_ang* math.pi/180.0 , 
                   delta_t_=delta_t, tread_=tread, start_x_=start_x, start_y_=start_y, start_ang_=start_ang* math.pi/180.0)
    traj_curve.calc_traj()
    d_pre = (end_x - start_x) - traj_curve.get_move_x()
    d_fol = (end_y - start_y) - traj_curve.get_move_y()
    
    t, omega = traj_curve.get_omega_degree(v)
    t, alpha = traj_curve.get_alpha_degree(v)
    t, omega_f, alpha_f, f = traj_curve.get_r_force(v, m, I)
    t, cent_acc = traj_curve.get_cent_acc(v)
    plt.axes().set_aspect('equal')
    
    # 前直線部分の軌跡描画
    plt.plot([start_x,start_x + d_pre],[start_y +tread/2 , start_y+tread/2]  ,color="red") 
    plt.plot([start_x,start_x + d_pre],[start_y, start_y]  ,color="red") 
    plt.plot([start_x,start_x + d_pre],[start_y -tread/2, start_y-tread/2  ]  ,color="red") 

    
    # カーブ部分の軌跡描画
    plt.plot(traj_curve.c_traj_x + d_pre, traj_curve.c_traj_y, color="blue")
    plt.plot(traj_curve.r_traj_x + d_pre, traj_curve.r_traj_y, color="blue")
    plt.plot(traj_curve.l_traj_x + d_pre, traj_curve.l_traj_y, color="blue")

    # 直後部分の軌跡描画
    plt.plot([traj_curve.get_end_x()+ d_pre +tread/2 , end_x+tread/2],[traj_curve.get_end_y(), end_y]  ,color="red") 
    plt.plot([traj_curve.get_end_x()+ d_pre , end_x],[traj_curve.get_end_y(), end_y]  ,color="red") 
    plt.plot([traj_curve.get_end_x()+ d_pre -tread/2, end_x-tread/2],[traj_curve.get_end_y(), end_y]  ,color="red") 

    plt.gca().xaxis.set_major_locator(tick.MultipleLocator(0.09))
    plt.gca().yaxis.set_major_locator(tick.MultipleLocator(0.09))
    plt.title("trajectory")
    plt.axis([-0.36, 0.36, -0.36, 0.36])

    plt.grid(True)
    for i in range(-2,4):
        for j in range(-2,4):
            plt.gca().add_patch( plt.Rectangle(xy=[i*0.18+ 0.09-0.006, j*0.18+0.09-0.006], width=0.012, height=0.012) )
    plt.show()

      
    print("d_pre x[m] " + str(d_pre))
    print("d_fol y[m] " + str(d_fol))
    print("total length[m] " + str(d_pre+d_fol+path_length__) )
    
    plt.subplot(4, 1, 1)
    plt.plot(t,omega)
    plt.title("omega [sec - deg/s] ")
    plt.grid()  

    plt.subplot(4, 1, 2)
    plt.plot(t,alpha)
    plt.title("alpha [sec - deg/ss] ")
    plt.grid()

    plt.subplot(4, 1, 3)
    plt.title("force tire [sec - N] ")
    plt.grid()
    plt.plot(t,omega_f, label="omega_f"  )
    plt.plot(t,alpha_f, label="alpha_f")
    plt.plot(t,f, label="total_f")
    plt.legend()

    plt.subplot(4, 1, 4)
    plt.title("centripetal acc [sec - m/ss] ")
    plt.grid()
    plt.plot(t,cent_acc, label="centripetal acc")
    
    plt.tight_layout() 
    plt.show()
    

interactive(children=(FloatSlider(value=2.6, description='shape_factor', max=8.0, min=0.1), FloatSlider(value=…

In [65]:
print(" ")
print("len(kappa) " + str(len(traj_cl_serch_90.kappa)))
print("s resolution[m] " + str(traj_cl_serch_90.delta_t))
print("----- 以下コピペ用 ----")
print(" ")

# kappaを書き出す。分解能はお好みで
for i,(s, kappa) in enumerate(zip(traj_cl_serch_90.s, traj_cl_serch_90.kappa)):
    if i % 1 == 0:
        print(str(kappa)+ "," )

 
len(kappa) 8999
s resolution[m] 1e-05
----- 以下コピペ用 ----
 
0.0,
0.0,
1.45147931617e-249,
6.31568146435e-187,
2.41868160391e-149,
2.74796844553e-124,
2.1664293285e-106,
5.73178181648e-93,
1.57776747517e-82,
3.54684413438e-74,
2.41617858189e-67,
1.19545111586e-61,
7.86810040322e-57,
1.0613812179e-52,
4.02905920313e-49,
5.45905072994e-46,
3.16641689706e-43,
9.0566369896e-41,
1.428221736e-38,
1.35781203947e-36,
8.36559469008e-35,
3.54369427183e-33,
1.08377562778e-31,
2.49247468816e-30,
4.46051892193e-29,
6.39400383898e-28,
7.52490002365e-27,
7.42585956073e-26,
6.25781613939e-25,
4.57494554757e-24,
2.94176747373e-23,
1.6838977953e-22,
8.6716225021e-22,
4.0552184571e-21,
1.73638618796e-20,
6.85783005218e-20,
2.51466833246e-19,
8.61140616637e-19,
2.76854422371e-18,
8.39593532595e-18,
2.41202899755e-17,
6.58977604171e-17,
1.7181305954e-16,
4.28867704205e-16,
1.02785864314e-15,
2.37158049689e-15,
5.28068575294e-15,
1.1372514642e-14,
2.37368631947e-14,
4.81068312835e-14,
9.48328370827e-14,
1.82

25.9072196505,
25.9066215845,
25.9060219026,
25.9054206028,
25.9048176836,
25.9042131432,
25.9036069799,
25.902999192,
25.9023897778,
25.9017787355,
25.9011660635,
25.9005517599,
25.8999358232,
25.8993182516,
25.8986990434,
25.8980781969,
25.8974557104,
25.8968315821,
25.8962058103,
25.8955783935,
25.8949493297,
25.8943186174,
25.8936862548,
25.8930522403,
25.892416572,
25.8917792484,
25.8911402676,
25.8904996281,
25.8898573281,
25.8892133658,
25.8885677397,
25.887920448,
25.8872714889,
25.8866208609,
25.8859685621,
25.8853145909,
25.8846589457,
25.8840016246,
25.883342626,
25.8826819482,
25.8820195896,
25.8813555483,
25.8806898228,
25.8800224112,
25.879353312,
25.8786825234,
25.8780100438,
25.8773358714,
25.8766600046,
25.8759824416,
25.8753031808,
25.8746222204,
25.8739395588,
25.8732551944,
25.8725691253,
25.87188135,
25.8711918666,
25.8705006736,
25.8698077693,
25.8691131519,
25.8684168198,
25.8677187713,
25.8670190047,
25.8663175183,
25.8656143104,
25.8649093793,
25.8642027235,
25

## クラシック最短90度

In [66]:
from pylab import rcParams
rcParams['figure.figsize'] = 7,7

# 設定パラメータ
tread = 0.03
delta_t = 0.00001
m = 0.02
I = 0.00003
start_x = -0.09
start_y = 0.00
end_x = 0.0
end_y = 0.09
target_ang = 90
start_ang = 0.0

shape_factor__ = 2.6
path_length__ = 0.09
v__ = 1.0
traj_cl_90 = CalcTraj()

@interact(shape_factor=(0.1, 8.0, 0.1), path_length=(0.01, 0.36, 0.01),  v = (0.01, 5.0, 0.01)  )
def h(shape_factor=shape_factor__, path_length=path_length__, v=v__):
    traj_curve = traj_cl_90
    traj_curve.set_param(shape_factor_=shape_factor, path_length_= path_length, target_ang_= target_ang* math.pi/180.0 , 
                   delta_t_=delta_t, tread_=tread, start_x_=start_x, start_y_=start_y, start_ang_=start_ang* math.pi/180.0)
    traj_curve.calc_traj()
    d_pre = (end_x - start_x) - traj_curve.get_move_x()
    d_fol = (end_y - start_y) - traj_curve.get_move_y()
    
    t, omega = traj_curve.get_omega_degree(v)
    t, alpha = traj_curve.get_alpha_degree(v)
    t, omega_f, alpha_f, f = traj_curve.get_r_force(v, m, I)
    t, cent_acc = traj_curve.get_cent_acc(v)
    plt.axes().set_aspect('equal')
    
    # 前直線部分の軌跡描画
    plt.plot([start_x,start_x + d_pre],[start_y +tread/2 , start_y+tread/2]  ,color="red") 
    plt.plot([start_x,start_x + d_pre],[start_y, start_y]  ,color="red") 
    plt.plot([start_x,start_x + d_pre],[start_y -tread/2, start_y-tread/2  ]  ,color="red") 

    
    # カーブ部分の軌跡描画
    plt.plot(traj_curve.c_traj_x + d_pre, traj_curve.c_traj_y, color="blue")
    plt.plot(traj_curve.r_traj_x + d_pre, traj_curve.r_traj_y, color="blue")
    plt.plot(traj_curve.l_traj_x + d_pre, traj_curve.l_traj_y, color="blue")

    # 直後部分の軌跡描画
    plt.plot([traj_curve.get_end_x()+ d_pre +tread/2 , end_x+tread/2],[traj_curve.get_end_y(), end_y]  ,color="red") 
    plt.plot([traj_curve.get_end_x()+ d_pre , end_x],[traj_curve.get_end_y(), end_y]  ,color="red") 
    plt.plot([traj_curve.get_end_x()+ d_pre -tread/2, end_x-tread/2],[traj_curve.get_end_y(), end_y]  ,color="red") 

    plt.gca().xaxis.set_major_locator(tick.MultipleLocator(0.09))
    plt.gca().yaxis.set_major_locator(tick.MultipleLocator(0.09))
    plt.title("trajectory")
    plt.axis([-0.36, 0.36, -0.36, 0.36])

    plt.grid(True)
    for i in range(-2,4):
        for j in range(-2,4):
            plt.gca().add_patch( plt.Rectangle(xy=[i*0.18+ 0.09-0.006, j*0.18+0.09-0.006], width=0.012, height=0.012) )
    plt.show()

      
    print("d_pre x[m] " + str(d_pre))
    print("d_fol y[m] " + str(d_fol))
    print("total length[m] " + str(d_pre+d_fol+path_length__) )
    
    plt.subplot(4, 1, 1)
    plt.plot(t,omega)
    plt.title("omega [sec - deg/s] ")
    plt.grid()  

    plt.subplot(4, 1, 2)
    plt.plot(t,alpha)
    plt.title("alpha [sec - deg/ss] ")
    plt.grid()

    plt.subplot(4, 1, 3)
    plt.title("force tire [sec - N] ")
    plt.grid()
    plt.plot(t,omega_f, label="omega_f"  )
    plt.plot(t,alpha_f, label="alpha_f")
    plt.plot(t,f, label="total_f")
    plt.legend()

    plt.subplot(4, 1, 4)
    plt.title("centripetal acc [sec - m/ss] ")
    plt.grid()
    plt.plot(t,cent_acc, label="centripetal acc")
    
    plt.tight_layout() 
    plt.show()
    

interactive(children=(FloatSlider(value=2.6, description='shape_factor', max=8.0, min=0.1), FloatSlider(value=…

In [67]:
print(" ")
print("len(kappa) " + str(len(traj_cl_90.kappa)))
print("s resolution[m] " + str(traj_cl_90.delta_t))
print("----- 以下コピペ用 ----")
print(" ")

# kappaを書き出す。分解能はお好みで
for i,(s, kappa) in enumerate(zip(traj_cl_90.s, traj_cl_90.kappa)):
    if i % 1 == 0:
        print(str(kappa)+ "," )

 
len(kappa) 8999
s resolution[m] 1e-05
----- 以下コピペ用 ----
 
0.0,
0.0,
1.45147931617e-249,
6.31568146435e-187,
2.41868160391e-149,
2.74796844553e-124,
2.1664293285e-106,
5.73178181648e-93,
1.57776747517e-82,
3.54684413438e-74,
2.41617858189e-67,
1.19545111586e-61,
7.86810040322e-57,
1.0613812179e-52,
4.02905920313e-49,
5.45905072994e-46,
3.16641689706e-43,
9.0566369896e-41,
1.428221736e-38,
1.35781203947e-36,
8.36559469008e-35,
3.54369427183e-33,
1.08377562778e-31,
2.49247468816e-30,
4.46051892193e-29,
6.39400383898e-28,
7.52490002365e-27,
7.42585956073e-26,
6.25781613939e-25,
4.57494554757e-24,
2.94176747373e-23,
1.6838977953e-22,
8.6716225021e-22,
4.0552184571e-21,
1.73638618796e-20,
6.85783005218e-20,
2.51466833246e-19,
8.61140616637e-19,
2.76854422371e-18,
8.39593532595e-18,
2.41202899755e-17,
6.58977604171e-17,
1.7181305954e-16,
4.28867704205e-16,
1.02785864314e-15,
2.37158049689e-15,
5.28068575294e-15,
1.1372514642e-14,
2.37368631947e-14,
4.81068312835e-14,
9.48328370827e-14,
1.82

25.5602369225,
25.5589244485,
25.5576097406,
25.5562927969,
25.5549736159,
25.5536521959,
25.5523285352,
25.5510026322,
25.5496744851,
25.5483440924,
25.5470114523,
25.5456765631,
25.5443394233,
25.5430000311,
25.5416583849,
25.5403144829,
25.5389683236,
25.5376199052,
25.536269226,
25.5349162845,
25.5335610789,
25.5322036075,
25.5308438688,
25.5294818609,
25.5281175822,
25.5267510311,
25.5253822059,
25.5240111049,
25.5226377263,
25.5212620687,
25.5198841302,
25.5185039092,
25.5171214041,
25.515736613,
25.5143495345,
25.5129601667,
25.511568508,
25.5101745568,
25.5087783113,
25.5073797699,
25.5059789309,
25.5045757925,
25.5031703532,
25.5017626113,
25.500352565,
25.4989402127,
25.4975255528,
25.4961085834,
25.494689303,
25.4932677098,
25.4918438022,
25.4904175785,
25.488989037,
25.487558176,
25.4861249939,
25.4846894889,
25.4832516593,
25.4818115036,
25.4803690199,
25.4789242067,
25.4774770622,
25.4760275847,
25.4745757725,
25.473121624,
25.4716651375,
25.4702063112,
25.4687451435,
25.

## クラシック大周り90度

In [68]:
from pylab import rcParams
rcParams['figure.figsize'] = 7,7

# 設定パラメータ
tread = 0.03
delta_t = 0.00001
m = 0.02
I = 0.00003
start_x = -0.18
start_y = 0.00
end_x = 0.0
end_y = 0.18
target_ang = 90
start_ang = 0.0

shape_factor__ = 2.6
path_length__ = 0.2
v__ = 1.0
traj_cl_L_90 = CalcTraj()

@interact(shape_factor=(0.1, 8.0, 0.1), path_length=(0.01, 0.36, 0.01),  v = (0.01, 5.0, 0.01)  )
def h(shape_factor=shape_factor__, path_length=path_length__, v=v__):
    traj_curve = traj_cl_L_90
    traj_curve.set_param(shape_factor_=shape_factor, path_length_= path_length, target_ang_= target_ang* math.pi/180.0 , 
                   delta_t_=delta_t, tread_=tread, start_x_=start_x, start_y_=start_y, start_ang_=start_ang* math.pi/180.0)
    traj_curve.calc_traj()
    d_pre = (end_x - start_x) - traj_curve.get_move_x()
    d_fol = (end_y - start_y) - traj_curve.get_move_y()
    
    t, omega = traj_curve.get_omega_degree(v)
    t, alpha = traj_curve.get_alpha_degree(v)
    t, omega_f, alpha_f, f = traj_curve.get_r_force(v, m, I)
    t, cent_acc = traj_curve.get_cent_acc(v)
    plt.axes().set_aspect('equal')
    
    # 前直線部分の軌跡描画
    plt.plot([start_x,start_x + d_pre],[start_y +tread/2 , start_y+tread/2]  ,color="red") 
    plt.plot([start_x,start_x + d_pre],[start_y, start_y]  ,color="red") 
    plt.plot([start_x,start_x + d_pre],[start_y -tread/2, start_y-tread/2  ]  ,color="red") 

    
    # カーブ部分の軌跡描画
    plt.plot(traj_curve.c_traj_x + d_pre, traj_curve.c_traj_y, color="blue")
    plt.plot(traj_curve.r_traj_x + d_pre, traj_curve.r_traj_y, color="blue")
    plt.plot(traj_curve.l_traj_x + d_pre, traj_curve.l_traj_y, color="blue")

    # 直後部分の軌跡描画
    plt.plot([traj_curve.get_end_x()+ d_pre +tread/2 , end_x+tread/2],[traj_curve.get_end_y(), end_y]  ,color="red") 
    plt.plot([traj_curve.get_end_x()+ d_pre , end_x],[traj_curve.get_end_y(), end_y]  ,color="red") 
    plt.plot([traj_curve.get_end_x()+ d_pre -tread/2, end_x-tread/2],[traj_curve.get_end_y(), end_y]  ,color="red") 

    plt.gca().xaxis.set_major_locator(tick.MultipleLocator(0.09))
    plt.gca().yaxis.set_major_locator(tick.MultipleLocator(0.09))
    plt.title("trajectory")
    plt.axis([-0.36, 0.36, -0.36, 0.36])

    plt.grid(True)
    for i in range(-2,4):
        for j in range(-2,4):
            plt.gca().add_patch( plt.Rectangle(xy=[i*0.18+ 0.09-0.006, j*0.18+0.09-0.006], width=0.012, height=0.012) )
    plt.show()

      
    print("d_pre x[m] " + str(d_pre))
    print("d_fol y[m] " + str(d_fol))
    print("total length[m] " + str(d_pre+d_fol+path_length__) )
    
    plt.subplot(4, 1, 1)
    plt.plot(t,omega)
    plt.title("omega [sec - deg/s] ")
    plt.grid()  

    plt.subplot(4, 1, 2)
    plt.plot(t,alpha)
    plt.title("alpha [sec - deg/ss] ")
    plt.grid()

    plt.subplot(4, 1, 3)
    plt.title("force tire [sec - N] ")
    plt.grid()
    plt.plot(t,omega_f, label="omega_f"  )
    plt.plot(t,alpha_f, label="alpha_f")
    plt.plot(t,f, label="total_f")
    plt.legend()

    plt.subplot(4, 1, 4)
    plt.title("centripetal acc [sec - m/ss] ")
    plt.grid()
    plt.plot(t,cent_acc, label="centripetal acc")
    
    plt.tight_layout() 
    plt.show()
    

interactive(children=(FloatSlider(value=2.6, description='shape_factor', max=8.0, min=0.1), FloatSlider(value=…

In [69]:
print(" ")
print("len(kappa) " + str(len(traj_cl_90.kappa)))
print("s resolution[m] " + str(traj_cl_90.delta_t))
print("----- 以下コピペ用 ----")
print(" ")

# kappaを書き出す。分解能はお好みで
for i,(s, kappa) in enumerate(zip(traj_cl_L_90.s, traj_cl_L_90.kappa)):
    if i % 1 == 0:
        print(str(kappa)+ "," )

 
len(kappa) 8999
s resolution[m] 1e-05
----- 以下コピペ用 ----
 
0.0,
0.0,
0.0,
0.0,
0.0,
9.45450613935e-278,
5.57426711623e-238,
3.75057164053e-208,
5.93719065936e-185,
2.15360580851e-166,
3.29822236737e-151,
1.48778448801e-138,
7.58559478107e-128,
1.14235857337e-118,
1.02779335663e-110,
9.3701219339e-104,
1.29651962088e-97,
3.72799187341e-92,
2.85476530231e-87,
7.09995363826e-83,
6.73446614069e-79,
2.7784354793e-75,
5.55783568701e-72,
5.90090066728e-69,
3.5879737179e-66,
1.33238903104e-63,
3.19196441386e-61,
5.17041481395e-59,
5.89678264373e-57,
4.9041641723e-55,
3.06652582682e-53,
1.48071899669e-51,
5.65259824556e-50,
1.74171663868e-48,
4.41200677983e-47,
9.33927914907e-46,
1.67622332636e-44,
2.58433615964e-43,
3.46291442765e-42,
4.07548664532e-41,
4.25290505631e-40,
3.969092502e-39,
3.33871879437e-38,
2.54933967308e-37,
1.77843307112e-36,
1.14016658285e-35,
6.75400327405e-35,
3.71503163087e-34,
1.9060881266e-33,
9.16043072618e-33,
4.13954210052e-32,
1.76520315154e-31,
7.12638271765e-31,

10.0728494386,
10.0738456546,
10.0748414502,
10.0758368256,
10.0768317808,
10.0778263161,
10.0788204314,
10.079814127,
10.0808074029,
10.0818002593,
10.0827926964,
10.0837847141,
10.0847763128,
10.0857674924,
10.0867582531,
10.0877485951,
10.0887385184,
10.0897280232,
10.0907171096,
10.0917057777,
10.0926940276,
10.0936818596,
10.0946692736,
10.0956562699,
10.0966428485,
10.0976290096,
10.0986147533,
10.0996000797,
10.100584989,
10.1015694812,
10.1025535565,
10.103537215,
10.1045204569,
10.1055032822,
10.1064856911,
10.1074676837,
10.1084492601,
10.1094304205,
10.1104111649,
10.1113914936,
10.1123714066,
10.113350904,
10.114329986,
10.1153086527,
10.1162869042,
10.1172647406,
10.1182421621,
10.1192191688,
10.1201957608,
10.1211719382,
10.1221477012,
10.1231230499,
10.1240979843,
10.1250725047,
10.1260466111,
10.1270203036,
10.1279935825,
10.1289664478,
10.1299388996,
10.130910938,
10.1318825633,
10.1328537754,
10.1338245745,
10.1347949608,
10.1357649344,
10.1367344954,
10.1377036438,
1

11.719754824,
11.7197562372,
11.7197576232,
11.719758982,
11.719760314,
11.7197616192,
11.7197628979,
11.7197641504,
11.7197653767,
11.7197665772,
11.719767752,
11.7197689013,
11.7197700253,
11.7197711244,
11.7197721985,
11.7197732481,
11.7197742732,
11.7197752742,
11.7197762511,
11.7197772044,
11.7197781341,
11.7197790404,
11.7197799237,
11.7197807841,
11.7197816219,
11.7197824373,
11.7197832304,
11.7197840016,
11.7197847511,
11.7197854791,
11.7197861859,
11.7197868716,
11.7197875366,
11.719788181,
11.7197888051,
11.7197894092,
11.7197899935,
11.7197905582,
11.7197911036,
11.7197916299,
11.7197921375,
11.7197926265,
11.7197930973,
11.71979355,
11.7197939849,
11.7197944024,
11.7197948027,
11.719795186,
11.7197955527,
11.719795903,
11.7197962372,
11.7197965555,
11.7197968584,
11.719797146,
11.7197974186,
11.7197976767,
11.7197979204,
11.71979815,
11.719798366,
11.7197985686,
11.7197987581,
11.7197989349,
11.7197990992,
11.7197992516,
11.7197993922,
11.7197995214,
11.7197996397,
11.71979

10.0336677804,
10.0326546463,
10.0316410867,
10.0306271014,
10.0296126904,
10.0285978534,
10.0275825904,
10.0265669012,
10.0255507858,
10.0245342439,
10.0235172755,
10.0224998805,
10.0214820586,
10.0204638098,
10.019445134,
10.0184260309,
10.0174065006,
10.0163865429,
10.0153661576,
10.0143453446,
10.0133241038,
10.0123024351,
10.0112803383,
10.0102578133,
10.00923486,
10.0082114782,
10.0071876679,
10.0061634289,
10.005138761,
10.0041136642,
10.0030881383,
10.0020621832,
10.0010357988,
10.0000089849,
9.9989817414,
9.99795406818,
9.99692596512,
9.99589743208,
9.99486846893,
9.99383907556,
9.99280925182,
9.9917789976,
9.99074831276,
9.98971719718,
9.98868565073,
9.98765367328,
9.98662126471,
9.98558842487,
9.98455515366,
9.98352145093,
9.98248731656,
9.98145275042,
9.98041775239,
9.97938232233,
9.97834646012,
9.97731016563,
9.97627343873,
9.97523627928,
9.97419868717,
9.97316066227,
9.97212220444,
9.97108331355,
9.97004398948,
9.96900423211,
9.96796404129,
9.9669234169,
9.96588235882,
9.

0.0319559652441,
0.0315976235012,
0.0312420871688,
0.0308893459381,
0.0305393894605,
0.030192207348,
0.0298477891738,
0.029506124472,
0.0291672027384,
0.0288310134307,
0.0284975459683,
0.0281667897333,
0.0278387340704,
0.0275133682875,
0.0271906816554,
0.026870663409,
0.0265533027469,
0.0262385888319,
0.0259265107918,
0.0256170577189,
0.025310218671,
0.0250059826717,
0.0247043387102,
0.0244052757423,
0.0241087826904,
0.023814848444,
0.0235234618598,
0.0232346117626,
0.022948286945,
0.0226644761683,
0.0223831681627,
0.0221043516276,
0.021828015232,
0.0215541476153,
0.0212827373869,
0.0210137731273,
0.0207472433882,
0.0204831366929,
0.020221441537,
0.0199621463883,
0.0197052396877,
0.0194507098494,
0.0191985452613,
0.0189487342855,
0.0187012652587,
0.018456126493,
0.0182133062755,
0.0179727928696,
0.0177345745151,
0.0174986394286,
0.017264975804,
0.0170335718131,
0.0168044156059,
0.0165774953112,
0.016352799037,
0.0161303148711,
0.0159100308812,
0.0156919351159,
0.0154760156051,
0.015262

## クラシック180度ターン
180度ターンは曲線長と角度を指定するだけでうまく終点と始点が結べないため注意。
カーブの終点が180度ターンの終了点になるように曲線長を2分探索で求めている。

In [72]:
from pylab import rcParams
rcParams['figure.figsize'] = 7,7

# 設定パラメータ
tread = 0.03
delta_t = 0.00001
m = 0.02
I = 0.00003
start_x = -0.18
start_y = 0.00
end_x = -0.18
end_y = 0.18
target_ang = 180
start_ang = 0.0

shape_factor__ = 2.6

v__ = 1.0
traj_cl_180 = CalcTraj()
d_pre__ = 0.01

@interact(shape_factor=(0.1, 8.0, 0.1), d_pre=(0.001, 0.3, 0.001) ,  v = (0.01, 5.0, 0.01)  )
def h(shape_factor=shape_factor__, d_pre = d_pre__, v=v__):
    
    traj_curve = traj_cl_180
    path_length_min = 0.1
    path_length_max = 1.0
    error_y = 1.0
    
    # 最小化するターゲット関数
    def calc_y_error(pl_):
        traj = CalcTraj()
        traj.set_param(shape_factor_=shape_factor, path_length_= pl_, target_ang_= target_ang* math.pi/180.0 , 
                   delta_t_=delta_t, tread_=tread, start_x_=start_x, start_y_=start_y, start_ang_=start_ang* math.pi/180.0)        
        traj.calc_traj()
        error_y_ = traj.get_move_y() - end_y
        return error_y_
    
    while abs(error_y) > 0.0001:
        pl = 0.5 * (path_length_min + path_length_max)
        error_y = calc_y_error(pl)
        path_length = pl
        if(error_y > 0):
            path_length_max = pl
        else:
            path_length_min = pl
        print("error_y: " + str(error_y))
        print("path_length: " + str(pl))
        print("min, max :",path_length_min, path_length_max)
        
    d_fol = d_pre
    
    traj_curve.set_param(shape_factor_=shape_factor, path_length_= path_length, target_ang_= target_ang* math.pi/180.0 , 
                   delta_t_=delta_t, tread_=tread, start_x_=start_x, start_y_=start_y, start_ang_=start_ang* math.pi/180.0)        
    traj_curve.calc_traj()
    
    t, omega = traj_curve.get_omega_degree(v)
    t, alpha = traj_curve.get_alpha_degree(v)
    t, omega_f, alpha_f, f = traj_curve.get_r_force(v, m, I)
    t, cent_acc = traj_curve.get_cent_acc(v)
    plt.axes().set_aspect('equal')
    
    # 前直線部分の軌跡描画
    plt.plot([start_x,start_x + d_pre],[start_y +tread/2 , start_y+tread/2]  ,color="red") 
    plt.plot([start_x,start_x + d_pre],[start_y, start_y]  ,color="red") 
    plt.plot([start_x,start_x + d_pre],[start_y -tread/2, start_y-tread/2  ]  ,color="red") 

    
    # カーブ部分の軌跡描画
    plt.plot(traj_curve.c_traj_x + d_pre, traj_curve.c_traj_y, color="blue")
    plt.plot(traj_curve.r_traj_x + d_pre, traj_curve.r_traj_y, color="blue")
    plt.plot(traj_curve.l_traj_x + d_pre, traj_curve.l_traj_y, color="blue")

    # 直後部分の軌跡描画
    plt.plot([traj_curve.get_end_x()+ d_pre , end_x],[traj_curve.get_end_y()+tread/2, traj_curve.get_end_y()+tread/2]  ,color="red") 
    plt.plot([traj_curve.get_end_x()+ d_pre , end_x],[traj_curve.get_end_y(), traj_curve.get_end_y()]  ,color="red") 
    plt.plot([traj_curve.get_end_x()+ d_pre , end_x],[traj_curve.get_end_y()-tread/2, traj_curve.get_end_y()-tread/2]  ,color="red") 


    plt.gca().xaxis.set_major_locator(tick.MultipleLocator(0.09))
    plt.gca().yaxis.set_major_locator(tick.MultipleLocator(0.09))
    plt.title("trajectory")
    plt.axis([-0.36, 0.36, -0.36, 0.36])

    plt.grid(True)
    for i in range(-2,4):
        for j in range(-2,4):
            plt.gca().add_patch( plt.Rectangle(xy=[i*0.18+ 0.09-0.006, j*0.18+0.09-0.006], width=0.012, height=0.012) )
    plt.show()

    print("move_y[m] " + str(traj_curve.get_move_y()) )  
    print("d_pre x[m] " + str(d_pre))
    print("d_fol y[m] " + str(d_fol))
    print("total length[m] " + str(d_pre+d_fol+path_length__) )
    
    plt.subplot(4, 1, 1)
    plt.plot(t,omega)
    plt.title("omega [sec - deg/s] ")
    plt.grid()  

    plt.subplot(4, 1, 2)
    plt.plot(t,alpha)
    plt.title("alpha [sec - deg/ss] ")
    plt.grid()

    plt.subplot(4, 1, 3)
    plt.title("force tire [sec - N] ")
    plt.grid()
    plt.plot(t,omega_f, label="omega_f"  )
    plt.plot(t,alpha_f, label="alpha_f")
    plt.plot(t,f, label="total_f")
    plt.legend()

    plt.subplot(4, 1, 4)
    plt.title("centripetal acc [sec - m/ss] ")
    plt.grid()
    plt.plot(t,cent_acc, label="centripetal acc")
    
    plt.tight_layout() 
    plt.show()
    

interactive(children=(FloatSlider(value=2.6, description='shape_factor', max=8.0, min=0.1), FloatSlider(value=…

In [73]:
print(" ")
print("len(kappa) " + str(len(traj_cl_180.kappa)))
print("s resolution[m] " + str(traj_cl_180.delta_t))
print("----- 以下コピペ用 ----")
print(" ")

# kappaを書き出す。分解能はお好みで
for i,(s, kappa) in enumerate(zip(traj_cl_180.s, traj_cl_180.kappa)):
    if i % 1 == 0:
        print(str(kappa)+ "," )

 
len(kappa) 38872
s resolution[m] 1e-05
----- 以下コピペ用 ----
 
0.0,
0.0,
0.0,
0.0,
0.0,
0.0,
0.0,
0.0,
0.0,
0.0,
1.75146905876e-294,
6.89093810351e-270,
4.45991451828e-249,
3.07161830518e-231,
8.85119646016e-216,
2.97976391008e-202,
2.56997156872e-190,
1.04587617858e-179,
3.25264635284e-170,
1.13688713352e-161,
6.10284939606e-154,
6.49586658239e-147,
1.69312343315e-140,
1.2884537677e-134,
3.31847317202e-129,
3.27783233897e-124,
1.38121376161e-119,
2.72020036117e-115,
2.70881425919e-111,
1.46019905161e-107,
4.52162066395e-104,
8.47219221582e-101,
1.00542960563e-97,
7.86797247351e-95,
4.20755947085e-92,
1.58720348115e-89,
4.34469131246e-87,
8.85125175412e-85,
1.37289601914e-82,
1.65477420929e-80,
1.57878325945e-78,
1.21238398553e-76,
7.60796459108e-75,
3.95536174773e-73,
1.72520513309e-71,
6.38565964043e-70,
2.02689699243e-68,
5.57044218441e-67,
1.33724331009e-65,
2.82695047762e-64,
5.30225724819e-63,
8.88456575988e-62,
1.33849318115e-60,
1.82375745097e-59,
2.25979332002e-58,
2.55935700294

5.4035216931,
5.40494966338,
5.40637743188,
5.40780499859,
5.40923236347,
5.41065952649,
5.41208648764,
5.41351324687,
5.41493980418,
5.41636615951,
5.41779231286,
5.4192182642,
5.42064401349,
5.42206956071,
5.42349490584,
5.42492004885,
5.4263449897,
5.42776972838,
5.42919426486,
5.43061859912,
5.43204273112,
5.43346666083,
5.43489038825,
5.43631391333,
5.43773723605,
5.43916035639,
5.44058327431,
5.44200598981,
5.44342850284,
5.44485081338,
5.44627292141,
5.44769482691,
5.44911652984,
5.45053803018,
5.45195932791,
5.453380423,
5.45480131542,
5.45622200515,
5.45764249217,
5.45906277645,
5.46048285796,
5.46190273669,
5.4633224126,
5.46474188567,
5.46616115587,
5.46758022319,
5.46899908759,
5.47041774906,
5.47183620756,
5.47325446308,
5.47467251559,
5.47609036506,
5.47750801147,
5.4789254548,
5.48034269502,
5.48175973211,
5.48317656604,
5.4845931968,
5.48600962435,
5.48742584867,
5.48884186975,
5.49025768755,
5.49167330205,
5.49308871323,
5.49450392106,
5.49591892552,
5.4973337266,
5.49

9.88451263446,
9.88514181703,
9.88577086948,
9.88639979185,
9.88702858414,
9.88765724637,
9.88828577858,
9.88891418077,
9.88954245297,
9.89017059519,
9.89079860746,
9.89142648981,
9.89205424223,
9.89268186477,
9.89330935743,
9.89393672024,
9.89456395321,
9.89519105638,
9.89581802975,
9.89644487334,
9.89707158719,
9.8976981713,
9.89832462569,
9.8989509504,
9.89957714543,
9.9002032108,
9.90082914654,
9.90145495267,
9.90208062921,
9.90270617617,
9.90333159357,
9.90395688145,
9.9045820398,
9.90520706867,
9.90583196806,
9.90645673799,
9.90708137849,
9.90770588958,
9.90833027127,
9.90895452358,
9.90957864654,
9.91020264017,
9.91082650447,
9.91145023949,
9.91207384522,
9.91269732171,
9.91332066895,
9.91394388698,
9.91456697581,
9.91518993547,
9.91581276597,
9.91643546733,
9.91705803958,
9.91768048273,
9.9183027968,
9.91892498181,
9.91954703779,
9.92016896475,
9.92079076271,
9.92141243169,
9.92203397172,
9.9226553828,
9.92327666497,
9.92389781823,
9.92451884262,
9.92513973815,
9.92576050484,
9

11.6539492332,
11.6541539589,
11.6543586188,
11.6545632128,
11.6547677409,
11.6549722032,
11.6551765997,
11.6553809303,
11.6555851952,
11.6557893942,
11.6559935275,
11.656197595,
11.6564015967,
11.6566055327,
11.6568094029,
11.6570132073,
11.6572169461,
11.6574206191,
11.6576242265,
11.6578277681,
11.658031244,
11.6582346543,
11.6584379989,
11.6586412779,
11.6588444912,
11.6590476388,
11.6592507209,
11.6594537373,
11.6596566882,
11.6598595734,
11.6600623931,
11.6602651471,
11.6604678357,
11.6606704586,
11.6608730161,
11.6610755079,
11.6612779343,
11.6614802952,
11.6616825906,
11.6618848204,
11.6620869848,
11.6622890837,
11.6624911172,
11.6626930852,
11.6628949878,
11.6630968249,
11.6632985967,
11.663500303,
11.6637019439,
11.6639035194,
11.6641050296,
11.6643064744,
11.6645078538,
11.6647091679,
11.6649104166,
11.6651116,
11.6653127181,
11.6655137709,
11.6657147584,
11.6659156807,
11.6661165376,
11.6663173293,
11.6665180557,
11.6667187169,
11.6669193128,
11.6671198435,
11.667320309,
11

12.0573980604,
12.057406716,
12.0574153529,
12.0574239712,
12.0574325707,
12.0574411516,
12.0574497139,
12.0574582575,
12.0574667826,
12.057475289,
12.0574837769,
12.0574922462,
12.0575006969,
12.0575091291,
12.0575175428,
12.057525938,
12.0575343148,
12.057542673,
12.0575510128,
12.0575593341,
12.0575676371,
12.0575759216,
12.0575841877,
12.0575924354,
12.0576006648,
12.0576088758,
12.0576170685,
12.0576252429,
12.057633399,
12.0576415368,
12.0576496563,
12.0576577575,
12.0576658405,
12.0576739053,
12.0576819519,
12.0576899803,
12.0576979905,
12.0577059826,
12.0577139565,
12.0577219123,
12.0577298499,
12.0577377695,
12.0577456709,
12.0577535543,
12.0577614197,
12.057769267,
12.0577770963,
12.0577849075,
12.0577927008,
12.0578004761,
12.0578082335,
12.0578159729,
12.0578236943,
12.0578313979,
12.0578390835,
12.0578467513,
12.0578544012,
12.0578620333,
12.0578696475,
12.0578772439,
12.0578848225,
12.0578923833,
12.0578999263,
12.0579074516,
12.0579149591,
12.057922449,
12.0579299211,
12

11.8903656659,
11.8902475792,
11.8901294412,
11.8900112517,
11.8898930109,
11.8897747185,
11.8896563748,
11.8895379796,
11.8894195329,
11.8893010348,
11.8891824852,
11.8890638841,
11.8889452315,
11.8888265274,
11.8887077718,
11.8885889647,
11.888470106,
11.8883511958,
11.8882322341,
11.8881132207,
11.8879941558,
11.8878750394,
11.8877558713,
11.8876366516,
11.8875173803,
11.8873980574,
11.8872786829,
11.8871592567,
11.8870397789,
11.8869202494,
11.8868006683,
11.8866810355,
11.886561351,
11.8864416148,
11.8863218269,
11.8862019872,
11.8860820959,
11.8859621528,
11.885842158,
11.8857221114,
11.8856020131,
11.885481863,
11.8853616611,
11.8852414074,
11.8851211019,
11.8850007446,
11.8848803355,
11.8847598746,
11.8846393618,
11.8845187972,
11.8843981807,
11.8842775124,
11.8841567921,
11.88403602,
11.883915196,
11.8837943201,
11.8836733923,
11.8835524125,
11.8834313809,
11.8833102972,
11.8831891617,
11.8830679741,
11.8829467346,
11.8828254431,
11.8827040997,
11.8825827042,
11.8824612567,
11

10.6844165779,
10.6839598299,
10.683502978,
10.6830460223,
10.6825889628,
10.6821317994,
10.6816745322,
10.681217161,
10.680759686,
10.6803021071,
10.6798444242,
10.6793866374,
10.6789287466,
10.6784707518,
10.678012653,
10.6775544503,
10.6770961435,
10.6766377326,
10.6761792178,
10.6757205988,
10.6752618757,
10.6748030486,
10.6743441173,
10.6738850819,
10.6734259423,
10.6729666986,
10.6725073507,
10.6720478986,
10.6715883423,
10.6711286817,
10.6706689169,
10.6702090479,
10.6697490745,
10.6692889969,
10.668828815,
10.6683685287,
10.6679081381,
10.6674476431,
10.6669870438,
10.6665263401,
10.666065532,
10.6656046194,
10.6651436024,
10.664682481,
10.6642212551,
10.6637599248,
10.6632984899,
10.6628369505,
10.6623753066,
10.6619135581,
10.6614517051,
10.6609897475,
10.6605276853,
10.6600655185,
10.6596032471,
10.659140871,
10.6586783903,
10.6582158049,
10.6577531148,
10.65729032,
10.6568274205,
10.6563644163,
10.6559013073,
10.6554380936,
10.654974775,
10.6545113517,
10.6540478235,
10.653

7.29222633069,
7.29110249462,
7.28997846034,
7.28885422785,
7.28772979712,
7.28660516813,
7.28548034087,
7.28435531531,
7.28323009144,
7.28210466923,
7.28097904867,
7.27985322973,
7.27872721241,
7.27760099667,
7.27647458251,
7.2753479699,
7.27422115882,
7.27309414925,
7.27196694118,
7.27083953459,
7.26971192945,
7.26858412575,
7.26745612346,
7.26632792258,
7.26519952308,
7.26407092494,
7.26294212814,
7.26181313267,
7.2606839385,
7.25955454561,
7.258424954,
7.25729516363,
7.25616517449,
7.25503498657,
7.25390459983,
7.25277401427,
7.25164322986,
7.25051224659,
7.24938106443,
7.24824968337,
7.24711810339,
7.24598632447,
7.24485434659,
7.24372216973,
7.24258979388,
7.24145721901,
7.24032444511,
7.23919147216,
7.23805830014,
7.23692492902,
7.2357913588,
7.23465758945,
7.23352362096,
7.2323894533,
7.23125508646,
7.23012052042,
7.22898575516,
7.22785079066,
7.2267156269,
7.22558026386,
7.22444470153,
7.22330893989,
7.22217297892,
7.2210368186,
7.21990045891,
7.21876389983,
7.21762714135,
7.2

0.763175749185,
0.761982770984,
0.760790556378,
0.759599106106,
0.758408420906,
0.757218501516,
0.756029348676,
0.754840963123,
0.753653345596,
0.752466496832,
0.751280417569,
0.750095108546,
0.748910570499,
0.747726804166,
0.746543810283,
0.745361589589,
0.74418014282,
0.742999470713,
0.741819574004,
0.74064045343,
0.739462109726,
0.73828454363,
0.737107755876,
0.735931747201,
0.73475651834,
0.733582070029,
0.732408403003,
0.731235517996,
0.730063415743,
0.72889209698,
0.72772156244,
0.726551812858,
0.725382848967,
0.724214671503,
0.723047281197,
0.721880678784,
0.720714864998,
0.71954984057,
0.718385606234,
0.717222162722,
0.716059510767,
0.714897651101,
0.713736584456,
0.712576311563,
0.711416833154,
0.710258149961,
0.709100262715,
0.707943172145,
0.706786878984,
0.70563138396,
0.704476687805,
0.703322791248,
0.702169695019,
0.701017399847,
0.699865906461,
0.698715215591,
0.697565327964,
0.69641624431,
0.695267965356,
0.69412049183,
0.692973824459,
0.691827963972,
0.690682911095,
0.

## クラシック 斜め 45度ターン

In [74]:
from pylab import rcParams
rcParams['figure.figsize'] = 7,7

# 設定パラメータ
tread = 0.03
delta_t = 0.00001
m = 0.02
I = 0.00003
start_x = -0.18
start_y = 0.00
end_x = 0.0
end_y = 0.09
target_ang = 45
start_ang = 0.0

shape_factor__ = 2.6
path_length__ = 0.12
v__ = 1.0
traj_cl_45 = CalcTraj()

@interact(shape_factor=(0.1, 8.0, 0.1), path_length=(0.01, 0.36, 0.01),  v = (0.01, 5.0, 0.01)  )
def h(shape_factor=shape_factor__, path_length=path_length__, v=v__):
    traj_curve = traj_cl_45
    traj_curve.set_param(shape_factor_=shape_factor, path_length_= path_length, target_ang_= target_ang* math.pi/180.0 , 
                   delta_t_=delta_t, tread_=tread, start_x_=start_x, start_y_=start_y, start_ang_=start_ang* math.pi/180.0)
    traj_curve.calc_traj()
    d_fol = ((end_y - start_y) - traj_curve.get_move_y()) * math.sqrt(2)
    d_pre = (end_x - start_x) - traj_curve.get_move_x() - d_fol/math.sqrt(2)
    
    
    t, omega = traj_curve.get_omega_degree(v)
    t, alpha = traj_curve.get_alpha_degree(v)
    t, omega_f, alpha_f, f = traj_curve.get_r_force(v, m, I)
    t, cent_acc = traj_curve.get_cent_acc(v)
    plt.axes().set_aspect('equal')
    
    # 前直線部分の軌跡描画
    plt.plot([start_x,start_x + d_pre],[start_y +tread/2 , start_y+tread/2]  ,color="red") 
    plt.plot([start_x,start_x + d_pre],[start_y, start_y]  ,color="red") 
    plt.plot([start_x,start_x + d_pre],[start_y -tread/2, start_y-tread/2  ]  ,color="red") 

    
    # カーブ部分の軌跡描画
    plt.plot(traj_curve.r_traj_x + d_pre, traj_curve.r_traj_y, color="blue")
    plt.plot(traj_curve.c_traj_x + d_pre, traj_curve.c_traj_y, color="blue")
    plt.plot(traj_curve.l_traj_x + d_pre, traj_curve.l_traj_y, color="blue")

    # 直後部分の軌跡描画
    plt.plot([traj_curve.r_traj_x[-1] + d_pre , traj_curve.r_traj_x[-1] + d_pre + d_fol/math.sqrt(2) ],[traj_curve.r_traj_y[-1], traj_curve.r_traj_y[-1]+ d_fol/math.sqrt(2)]  ,color="red") 
    plt.plot([traj_curve.get_end_x()+ d_pre , end_x],[traj_curve.get_end_y(), end_y]  ,color="red") 
    plt.plot([traj_curve.l_traj_x[-1] + d_pre , traj_curve.l_traj_x[-1] + d_pre + d_fol/math.sqrt(2) ],[traj_curve.l_traj_y[-1], traj_curve.l_traj_y[-1]+ d_fol/math.sqrt(2)]  ,color="red") 

    
    plt.gca().xaxis.set_major_locator(tick.MultipleLocator(0.09))
    plt.gca().yaxis.set_major_locator(tick.MultipleLocator(0.09))
    plt.title("trajectory")
    plt.axis([-0.36, 0.36, -0.36, 0.36])

    plt.grid(True)
    for i in range(-2,4):
        for j in range(-2,4):
            plt.gca().add_patch( plt.Rectangle(xy=[i*0.18+ 0.09-0.006, j*0.18+0.09-0.006], width=0.012, height=0.012) )
    plt.show()

      
    print("d_pre x[m] " + str(d_pre))
    print("d_fol y[m] " + str(d_fol))
    print("total length[m] " + str(d_pre+d_fol+path_length__) )
    
    plt.subplot(4, 1, 1)
    plt.plot(t,omega)
    plt.title("omega [sec - deg/s] ")
    plt.grid()  

    plt.subplot(4, 1, 2)
    plt.plot(t,alpha)
    plt.title("alpha [sec - deg/ss] ")
    plt.grid()

    plt.subplot(4, 1, 3)
    plt.title("force tire [sec - N] ")
    plt.grid()
    plt.plot(t,omega_f, label="omega_f"  )
    plt.plot(t,alpha_f, label="alpha_f")
    plt.plot(t,f, label="total_f")
    plt.legend()

    plt.subplot(4, 1, 4)
    plt.title("centripetal acc [sec - m/ss] ")
    plt.grid()
    plt.plot(t,cent_acc, label="centripetal acc")
    
    plt.tight_layout() 
    plt.show()
    

interactive(children=(FloatSlider(value=2.6, description='shape_factor', max=8.0, min=0.1), FloatSlider(value=…

In [75]:
print(" ")
print("len(kappa) " + str(len(traj_cl_45.kappa)))
print("s resolution[m] " + str(traj_cl_45.delta_t))
print("----- 以下コピペ用 ----")
print(" ")

# kappaを書き出す。分解能はお好みで
for i,(s, kappa) in enumerate(zip(traj_cl_45.s, traj_cl_45.kappa)):
    if i % 1 == 0:
        print(str(kappa)+ "," )

 
len(kappa) 11999
s resolution[m] 1e-05
----- 以下コピペ用 ----
 
0.0,
0.0,
0.0,
5.44304743564e-250,
7.02636172988e-200,
1.79467150709e-166,
1.30709080928e-142,
1.03048816707e-124,
8.56494463859e-111,
1.17075796828e-99,
1.51191293606e-90,
5.91662803188e-83,
1.57230016639e-76,
5.04910692966e-71,
2.98997809825e-66,
4.48294168449e-62,
2.16857685867e-58,
4.08680347691e-55,
3.48202945951e-52,
1.51089720117e-49,
3.67667231652e-47,
5.42930976392e-45,
5.19277208114e-43,
3.3962388711e-41,
1.58982090244e-39,
5.53616063629e-38,
1.48202755927e-36,
3.13709800878e-35,
5.37988911496e-34,
7.63369227182e-33,
9.12798712749e-32,
9.34678008058e-31,
8.31225193766e-30,
6.50051426519e-29,
4.51995702057e-28,
2.82183750887e-27,
1.59563847515e-26,
8.23637584517e-26,
3.90825767885e-25,
1.71560458034e-24,
7.00668029339e-24,
2.67612870459e-23,
9.60354426236e-23,
3.25185843829e-22,
1.04301101215e-21,
3.18008501987e-21,
9.2466809397e-21,
2.57168626957e-20,
6.85989747319e-20,
1.75943301451e-19,
4.34896528949e-19,
1.038204

9.72556541725,
9.72571135673,
9.7258569741,
9.72600226962,
9.72614724358,
9.72629189625,
9.72643622792,
9.72658023886,
9.72672392934,
9.72686729965,
9.72701035007,
9.72715308087,
9.72729549233,
9.72743758473,
9.72757935835,
9.72772081347,
9.72786195036,
9.72800276931,
9.7281432706,
9.7282834545,
9.7284233213,
9.72856287127,
9.72870210469,
9.72884102185,
9.72897962302,
9.72911790848,
9.72925587851,
9.7293935334,
9.72953087342,
9.72966789885,
9.72980460998,
9.72994100708,
9.73007709044,
9.73021286033,
9.73034831704,
9.73048346084,
9.73061829203,
9.73075281087,
9.73088701766,
9.73102091267,
9.73115449618,
9.73128776848,
9.73142072984,
9.73155338056,
9.73168572091,
9.73181775117,
9.73194947162,
9.73208088256,
9.73221198425,
9.73234277699,
9.73247326105,
9.73260343672,
9.73273330429,
9.73286286402,
9.73299211622,
9.73312106116,
9.73324969912,
9.73337803038,
9.73350605524,
9.73363377398,
9.73376118687,
9.7338882942,
9.73401509627,
9.73414159334,
9.73426778571,
9.73439367365,
9.73451925747,
9

8.36728984562,
8.36588665055,
8.3644824723,
8.36307731039,
8.36167116434,
8.36026403365,
8.35885591785,
8.35744681643,
8.35603672892,
8.35462565483,
8.35321359366,
8.35180054494,
8.35038650817,
8.34897148285,
8.34755546851,
8.34613846465,
8.34472047079,
8.34330148643,
8.34188151108,
8.34046054425,
8.33903858545,
8.3376156342,
8.33619168999,
8.33476675233,
8.33334082075,
8.33191389473,
8.33048597379,
8.32905705744,
8.32762714519,
8.32619623653,
8.32476433099,
8.32333142805,
8.32189752724,
8.32046262805,
8.31902672999,
8.31758983256,
8.31615193528,
8.31471303764,
8.31327313915,
8.31183223931,
8.31039033763,
8.30894743361,
8.30750352675,
8.30605861656,
8.30461270254,
8.3031657842,
8.30171786102,
8.30026893253,
8.29881899821,
8.29736805756,
8.2959161101,
8.29446315532,
8.29300919273,
8.29155422181,
8.29009824208,
8.28864125303,
8.28718325416,
8.28572424497,
8.28426422496,
8.28280319363,
8.28134115047,
8.279878095,
8.27841402669,
8.27694894506,
8.27548284959,
8.27401573979,
8.27254761515,
8

## クラシック 斜め 135度ターン

In [84]:
from pylab import rcParams
rcParams['figure.figsize'] = 7,7

# 設定パラメータ
tread = 0.03
delta_t = 0.00001
m = 0.02
I = 0.00003
start_x = -0.18
start_y = 0.00
end_x = -0.09
end_y = 0.18
target_ang = 135
start_ang = 0.0

shape_factor__ = 2.6
path_length__ = 0.25
v__ = 1.0
traj_cl_135 = CalcTraj()

@interact(shape_factor=(0.1, 8.0, 0.1), path_length=(0.01, 0.36, 0.01),  v = (0.01, 5.0, 0.01)  )
def h(shape_factor=shape_factor__, path_length=path_length__, v=v__):
    traj_curve = traj_cl_135
    traj_curve.set_param(shape_factor_=shape_factor, path_length_= path_length, target_ang_= target_ang* math.pi/180.0 , 
                   delta_t_=delta_t, tread_=tread, start_x_=start_x, start_y_=start_y, start_ang_=start_ang* math.pi/180.0)
    traj_curve.calc_traj()
    d_fol = ((end_y - start_y) - traj_curve.get_move_y()) * math.sqrt(2)
    d_pre = (end_x - start_x) - traj_curve.get_move_x() + d_fol/math.sqrt(2)
    
    
    t, omega = traj_curve.get_omega_degree(v)
    t, alpha = traj_curve.get_alpha_degree(v)
    t, omega_f, alpha_f, f = traj_curve.get_r_force(v, m, I)
    t, cent_acc = traj_curve.get_cent_acc(v)
    plt.axes().set_aspect('equal')
    
    # 前直線部分の軌跡描画
    plt.plot([start_x,start_x + d_pre],[start_y +tread/2 , start_y+tread/2]  ,color="red") 
    plt.plot([start_x,start_x + d_pre],[start_y, start_y]  ,color="red") 
    plt.plot([start_x,start_x + d_pre],[start_y -tread/2, start_y-tread/2  ]  ,color="red") 

    
    # カーブ部分の軌跡描画
    plt.plot(traj_curve.r_traj_x + d_pre, traj_curve.r_traj_y, color="blue")
    plt.plot(traj_curve.c_traj_x + d_pre, traj_curve.c_traj_y, color="blue")
    plt.plot(traj_curve.l_traj_x + d_pre, traj_curve.l_traj_y, color="blue")

    # 直後部分の軌跡描画
    plt.plot([traj_curve.r_traj_x[-1] + d_pre , traj_curve.r_traj_x[-1] + d_pre - d_fol/math.sqrt(2) ],[traj_curve.r_traj_y[-1], traj_curve.r_traj_y[-1]+ d_fol/math.sqrt(2)]  ,color="red") 
    plt.plot([traj_curve.get_end_x()+ d_pre , end_x],[traj_curve.get_end_y(), end_y]  ,color="red") 
    plt.plot([traj_curve.l_traj_x[-1] + d_pre , traj_curve.l_traj_x[-1] + d_pre - d_fol/math.sqrt(2) ],[traj_curve.l_traj_y[-1], traj_curve.l_traj_y[-1]+ d_fol/math.sqrt(2)]  ,color="red") 

    
    plt.gca().xaxis.set_major_locator(tick.MultipleLocator(0.09))
    plt.gca().yaxis.set_major_locator(tick.MultipleLocator(0.09))
    plt.title("trajectory")
    plt.axis([-0.36, 0.36, -0.36, 0.36])

    plt.grid(True)
    for i in range(-2,4):
        for j in range(-2,4):
            plt.gca().add_patch( plt.Rectangle(xy=[i*0.18+ 0.09-0.006, j*0.18+0.09-0.006], width=0.012, height=0.012) )
    plt.show()

      
    print("d_pre x[m] " + str(d_pre))
    print("d_fol y[m] " + str(d_fol))
    print("total length[m] " + str(d_pre+d_fol+path_length__) )
    
    plt.subplot(4, 1, 1)
    plt.plot(t,omega)
    plt.title("omega [sec - deg/s] ")
    plt.grid()  

    plt.subplot(4, 1, 2)
    plt.plot(t,alpha)
    plt.title("alpha [sec - deg/ss] ")
    plt.grid()

    plt.subplot(4, 1, 3)
    plt.title("force tire [sec - N] ")
    plt.grid()
    plt.plot(t,omega_f, label="omega_f"  )
    plt.plot(t,alpha_f, label="alpha_f")
    plt.plot(t,f, label="total_f")
    plt.legend()

    plt.subplot(4, 1, 4)
    plt.title("centripetal acc [sec - m/ss] ")
    plt.grid()
    plt.plot(t,cent_acc, label="centripetal acc")
    
    plt.tight_layout() 
    plt.show()

interactive(children=(FloatSlider(value=2.6, description='shape_factor', max=8.0, min=0.1), FloatSlider(value=…

In [85]:
print(" ")
print("len(kappa) " + str(len(traj_cl_135.kappa)))
print("s resolution[m] " + str(traj_cl_135.delta_t))
print("----- 以下コピペ用 ----")
print(" ")

# kappaを書き出す。分解能はお好みで
for i,(s, kappa) in enumerate(zip(traj_cl_135.s, traj_cl_135.kappa)):
    if i % 1 == 0:
        print(str(kappa)+ "," )

 
len(kappa) 24999
s resolution[m] 1e-05
----- 以下コピペ用 ----
 
0.0,
0.0,
0.0,
0.0,
0.0,
0.0,
1.47755898695e-297,
2.84731522995e-260,
2.84310943064e-231,
4.50068596863e-208,
4.31195751306e-189,
2.83467454469e-173,
6.86781889003e-160,
2.03745702104e-148,
1.78534138561e-138,
8.94384333387e-130,
4.24443720972e-122,
2.82613926546e-115,
3.6001159781e-109,
1.12441463207e-103,
1.05254260085e-98,
3.48028435355e-94,
4.65586181201e-90,
2.82175596949e-86,
8.51994436591e-83,
1.38888927332e-79,
1.30904623228e-76,
7.5647188101e-74,
2.81941221651e-71,
7.08108080074e-69,
1.24510278956e-66,
1.5848616489e-64,
1.5038839322e-62,
1.09176626402e-60,
6.20449777674e-59,
2.81712903958e-57,
1.0407190893e-55,
3.17949797167e-54,
8.15132339452e-53,
1.77686279603e-51,
3.33265447077e-50,
5.4362676796e-49,
7.78780804835e-48,
9.88504603773e-47,
1.12071349789e-45,
1.14330396826e-44,
1.05659010714e-43,
8.90044524156e-43,
6.87291447527e-42,
4.89058397439e-41,
3.22225469033e-40,
1.97454249877e-39,
1.12996028473e-38,
6.061784

9.68246427553,
9.68416789132,
9.68587101043,
9.68757363299,
9.68927575908,
9.69097738883,
9.69267852233,
9.69437915968,
9.696079301,
9.69777894639,
9.69947809596,
9.7011767498,
9.70287490803,
9.70457257075,
9.70626973807,
9.70796641009,
9.70966258692,
9.71135826866,
9.71305345541,
9.71474814729,
9.7164423444,
9.71813604685,
9.71982925473,
9.72152196816,
9.72321418724,
9.72490591207,
9.72659714277,
9.72828787943,
9.72997812217,
9.73166787108,
9.73335712628,
9.73504588787,
9.73673415595,
9.73842193063,
9.74010921202,
9.74179600021,
9.74348229533,
9.74516809746,
9.74685340673,
9.74853822322,
9.75022254706,
9.75190637834,
9.75358971716,
9.75527256365,
9.75695491789,
9.75863678,
9.76031815008,
9.76199902824,
9.76367941458,
9.76535930921,
9.76703871223,
9.76871762376,
9.77039604389,
9.77207397272,
9.77375141038,
9.77542835695,
9.77710481256,
9.77878077729,
9.78045625127,
9.78213123459,
9.78380572735,
9.78547972968,
9.78715324166,
9.78882626341,
9.79049879503,
9.79217083663,
9.79384238831,
9.

13.6920462133,
13.6923645379,
13.6926826898,
13.6930006691,
13.6933184758,
13.69363611,
13.6939535717,
13.694270861,
13.6945879779,
13.6949049225,
13.6952216947,
13.6955382947,
13.6958547225,
13.6961709781,
13.6964870615,
13.6968029729,
13.6971187122,
13.6974342795,
13.6977496749,
13.6980648984,
13.69837995,
13.6986948297,
13.6990095377,
13.6993240739,
13.6996384384,
13.6999526313,
13.7002666526,
13.7005805023,
13.7008941805,
13.7012076872,
13.7015210224,
13.7018341863,
13.7021471788,
13.70246,
13.7027726499,
13.7030851286,
13.7033974362,
13.7037095726,
13.7040215379,
13.7043333321,
13.7046449554,
13.7049564077,
13.705267689,
13.7055787995,
13.7058897392,
13.7062005081,
13.7065111062,
13.7068215336,
13.7071317904,
13.7074418765,
13.7077517921,
13.7080615372,
13.7083711117,
13.7086805158,
13.7089897495,
13.7092988129,
13.7096077059,
13.7099164287,
13.7102249813,
13.7105333636,
13.7108415758,
13.711149618,
13.71145749,
13.7117651921,
13.7120727242,
13.7123800863,
13.7126872786,
13.712994

13.9906316919,
13.9905162432,
13.9904006817,
13.9902850074,
13.9901692203,
13.9900533203,
13.9899373073,
13.9898211814,
13.9897049426,
13.9895885906,
13.9894721256,
13.9893555474,
13.9892388561,
13.9891220515,
13.9890051337,
13.9888881026,
13.9887709581,
13.9886537002,
13.9885363289,
13.9884188441,
13.9883012457,
13.9881835338,
13.9880657083,
13.9879477692,
13.9878297163,
13.9877115497,
13.9875932693,
13.9874748751,
13.9873563671,
13.9872377451,
13.9871190091,
13.9870001592,
13.9868811952,
13.9867621171,
13.9866429249,
13.9865236185,
13.9864041979,
13.9862846631,
13.9861650139,
13.9860452504,
13.9859253725,
13.9858053802,
13.9856852734,
13.9855650521,
13.9854447163,
13.9853242658,
13.9852037007,
13.9850830209,
13.9849622263,
13.984841317,
13.9847202929,
13.9845991539,
13.9844779,
13.9843565311,
13.9842350473,
13.9841134484,
13.9839917345,
13.9838699054,
13.9837479612,
13.9836259017,
13.9835037271,
13.9833814371,
13.9832590318,
13.9831365111,
13.983013875,
13.9828911234,
13.9827682563,


11.8943970691,
11.8933757893,
11.8923541711,
11.8913322145,
11.8903099194,
11.8892872856,
11.8882643132,
11.887241002,
11.8862173519,
11.8851933629,
11.8841690349,
11.8831443677,
11.8821193614,
11.8810940158,
11.8800683309,
11.8790423066,
11.8780159428,
11.8769892393,
11.8759621962,
11.8749348134,
11.8739070907,
11.8728790282,
11.8718506256,
11.870821883,
11.8697928002,
11.8687633772,
11.8677336139,
11.8667035102,
11.865673066,
11.8646422812,
11.8636111559,
11.8625796898,
11.8615478829,
11.8605157351,
11.8594832463,
11.8584504166,
11.8574172457,
11.8563837335,
11.8553498801,
11.8543156854,
11.8532811491,
11.8522462714,
11.851211052,
11.850175491,
11.8491395881,
11.8481033435,
11.8470667568,
11.8460298282,
11.8449925574,
11.8439549445,
11.8429169893,
11.8418786918,
11.8408400518,
11.8398010693,
11.8387617443,
11.8377220765,
11.836682066,
11.8356417127,
11.8346010165,
11.8335599772,
11.8325185949,
11.8314768694,
11.8304348007,
11.8293923886,
11.8283496331,
11.8273065342,
11.8262630917,
1

1.55402876897,
1.55139591445,
1.54876434948,
1.54613407699,
1.54350509989,
1.54087742107,
1.53825104347,
1.53562596999,
1.53300220355,
1.53037974708,
1.52775860351,
1.52513877575,
1.52252026674,
1.5199030794,
1.51728721668,
1.51467268152,
1.51205947683,
1.50944760559,
1.50683707071,
1.50422787516,
1.50162002187,
1.49901351381,
1.49640835392,
1.49380454516,
1.4912020905,
1.48860099288,
1.48600125527,
1.48340288065,
1.48080587197,
1.47821023221,
1.47561596434,
1.47302307133,
1.47043155617,
1.46784142183,
1.46525267129,
1.46266530754,
1.46007933357,
1.45749475237,
1.45491156692,
1.45232978022,
1.44974939527,
1.44717041506,
1.4445928426,
1.44201668088,
1.43944193292,
1.43686860172,
1.43429669029,
1.43172620165,
1.4291571388,
1.42658950476,
1.42402330256,
1.42145853521,
1.41889520574,
1.41633331717,
1.41377287253,
1.41121387484,
1.40865632715,
1.40610023249,
1.40354559389,
1.40099241438,
1.39844069702,
1.39589044485,
1.3933416609,
1.39079434823,
1.38824850989,
1.38570414892,
1.38316126839,


## クラシック 斜め 90度ターン

In [120]:
from pylab import rcParams
rcParams['figure.figsize'] = 7,7

# 設定パラメータ
tread = 0.03
delta_t = 0.00001
m = 0.02
I = 0.00003
start_x = -0.09
start_y = 0.00
end_x = -0.09
end_y = 0.18
target_ang = 90
start_ang = 45

shape_factor__ = 2.6
path_length__ = 0.15
v__ = 1.0
traj_cl_D_90 = CalcTraj()

@interact(shape_factor=(0.1, 8.0, 0.1), path_length=(0.01, 0.36, 0.01),  v = (0.01, 5.0, 0.01)  )
def h(shape_factor=shape_factor__, path_length=path_length__, v=v__):
    traj_curve = traj_cl_D_90
    traj_curve.set_param(shape_factor_=shape_factor, path_length_= path_length, target_ang_= target_ang* math.pi/180.0 , 
                   delta_t_=delta_t, tread_=tread, start_x_=start_x, start_y_=start_y, start_ang_=start_ang* math.pi/180.0)
    traj_curve.calc_traj()
    d_fol = ((end_y - start_y) - traj_curve.get_move_y()) /(math.sqrt(2))
    d_pre = d_fol
    
    
    t, omega = traj_curve.get_omega_degree(v)
    t, alpha = traj_curve.get_alpha_degree(v)
    t, omega_f, alpha_f, f = traj_curve.get_r_force(v, m, I)
    t, cent_acc = traj_curve.get_cent_acc(v)
    plt.axes().set_aspect('equal')
    
    # 前直線部分の軌跡描画
    #plt.plot([start_x,start_x + d_pre],[start_y +tread/2 , start_y+tread/2]  ,color="red") 
    plt.plot([traj_curve.r_traj_x[0]  ,traj_curve.r_traj_x[0] + d_pre/math.sqrt(2)],[traj_curve.r_traj_y[0], traj_curve.r_traj_y[0] + d_pre/math.sqrt(2)]  ,color="red") 
    plt.plot([start_x,traj_curve.c_traj_x[0] + d_pre/math.sqrt(2)],[start_y, traj_curve.c_traj_y[0] + d_pre/math.sqrt(2)]  ,color="red") 
    plt.plot([traj_curve.l_traj_x[0], traj_curve.l_traj_x[0] + d_pre/math.sqrt(2)],[traj_curve.l_traj_y[0], traj_curve.l_traj_y[0] + d_pre/math.sqrt(2)]  ,color="red") 

    #plt.plot([start_x,start_x + d_pre],[start_y -tread/2, start_y-tread/2  ]  ,color="red") 

    
    # カーブ部分の軌跡描画
    plt.plot(traj_curve.r_traj_x + d_pre/math.sqrt(2), traj_curve.r_traj_y + d_pre/math.sqrt(2), color="blue")
    plt.plot(traj_curve.c_traj_x + d_pre/math.sqrt(2), traj_curve.c_traj_y + d_pre/math.sqrt(2) , color="blue")
    plt.plot(traj_curve.l_traj_x + d_pre/math.sqrt(2), traj_curve.l_traj_y + d_pre/math.sqrt(2), color="blue")

    # 直後部分の軌跡描画
    plt.plot([traj_curve.r_traj_x[-1] + d_pre/math.sqrt(2) , traj_curve.r_traj_x[-1]  ],[traj_curve.r_traj_y[-1]+ d_pre/math.sqrt(2), traj_curve.r_traj_y[-1]+ 2*d_fol/math.sqrt(2)]  ,color="red") 
    plt.plot([traj_curve.get_end_x() + d_pre/math.sqrt(2) , end_x],[traj_curve.get_end_y() + d_pre/math.sqrt(2), end_y]  ,color="red") 
    plt.plot([traj_curve.l_traj_x[-1] + d_pre/math.sqrt(2) , traj_curve.l_traj_x[-1]  ],[traj_curve.l_traj_y[-1]+ d_pre/math.sqrt(2) , traj_curve.l_traj_y[-1]+ 2*d_fol/math.sqrt(2)]  ,color="red") 

    
    plt.gca().xaxis.set_major_locator(tick.MultipleLocator(0.09))
    plt.gca().yaxis.set_major_locator(tick.MultipleLocator(0.09))
    plt.title("trajectory")
    plt.axis([-0.36, 0.36, -0.36, 0.36])

    plt.grid(True)
    for i in range(-2,4):
        for j in range(-2,4):
            plt.gca().add_patch( plt.Rectangle(xy=[i*0.18+ 0.09-0.006, j*0.18+0.09-0.006], width=0.012, height=0.012) )
    plt.show()

      
    print("d_pre x[m] " + str(d_pre))
    print("d_fol y[m] " + str(d_fol))
    print("total length[m] " + str(d_pre+d_fol+path_length__) )
    
    plt.subplot(4, 1, 1)
    plt.plot(t,omega)
    plt.title("omega [sec - deg/s] ")
    plt.grid()  

    plt.subplot(4, 1, 2)
    plt.plot(t,alpha)
    plt.title("alpha [sec - deg/ss] ")
    plt.grid()

    plt.subplot(4, 1, 3)
    plt.title("force tire [sec - N] ")
    plt.grid()
    plt.plot(t,omega_f, label="omega_f"  )
    plt.plot(t,alpha_f, label="alpha_f")
    plt.plot(t,f, label="total_f")
    plt.legend()

    plt.subplot(4, 1, 4)
    plt.title("centripetal acc [sec - m/ss] ")
    plt.grid()
    plt.plot(t,cent_acc, label="centripetal acc")
    
    plt.tight_layout() 
    plt.show()

interactive(children=(FloatSlider(value=2.6, description='shape_factor', max=8.0, min=0.1), FloatSlider(value=…

In [121]:
print(" ")
print("len(kappa) " + str(len(traj_cl_D_90.kappa)))
print("s resolution[m] " + str(traj_cl_D_90.delta_t))
print("----- 以下コピペ用 ----")
print(" ")

# kappaを書き出す。分解能はお好みで
for i,(s, kappa) in enumerate(zip(traj_cl_D_90.s, traj_cl_D_90.kappa)):
    if i % 1 == 0:
        print(str(kappa)+ "," )

 
len(kappa) 14999
s resolution[m] 1e-05
----- 以下コピペ用 ----
 
0.0,
0.0,
0.0,
2.00145394454e-312,
8.70887589703e-250,
5.00076218737e-208,
3.36467573037e-178,
7.90443439755e-156,
1.98371265068e-138,
1.64878106732e-124,
4.03639440224e-113,
1.24934959119e-103,
1.34049902157e-95,
1.02475823697e-88,
9.46660485101e-83,
1.57061281559e-77,
6.33635986594e-73,
7.86786755637e-69,
3.62179560831e-65,
7.17270669518e-62,
6.8938864186e-59,
3.54880622136e-56,
1.06146732643e-53,
1.97429199559e-51,
2.41743552188e-49,
2.04496814801e-47,
1.24523721988e-45,
5.65389713963e-44,
1.97315181872e-42,
5.43398219376e-41,
1.20829986963e-39,
2.21329621653e-38,
3.39911956411e-37,
4.4453787838e-36,
5.01935681405e-35,
4.95337550783e-34,
4.31927084464e-33,
3.36061354069e-32,
2.35360420205e-31,
1.49548481289e-30,
8.68274319447e-30,
4.63613842667e-29,
2.28990860225e-28,
1.05183915273e-27,
4.51494001419e-27,
1.8190530161e-26,
6.9069348614e-26,
2.48074189231e-25,
8.45691973398e-25,
2.74496732854e-24,
8.5076529915e-24,
2.524566

12.7875835683,
12.789706709,
12.7918287136,
12.7939495824,
12.7960693158,
12.7981879144,
12.8003053785,
12.8024217087,
12.8045369054,
12.806650969,
12.8087639,
12.8108756988,
12.8129863659,
12.8150959018,
12.8172043068,
12.8193115815,
12.8214177263,
12.8235227416,
12.8256266279,
12.8277293856,
12.8298310152,
12.8319315171,
12.8340308919,
12.8361291398,
12.8382262615,
12.8403222572,
12.8424171276,
12.8445108729,
12.8466034938,
12.8486949905,
12.8507853637,
12.8528746136,
12.8549627408,
12.8570497457,
12.8591356288,
12.8612203905,
12.8633040312,
12.8653865514,
12.8674679515,
12.8695482321,
12.8716273934,
12.8737054361,
12.8757823604,
12.877858167,
12.8799328561,
12.8820064283,
12.884078884,
12.8861502237,
12.8882204477,
12.8902895566,
12.8923575507,
12.8944244306,
12.8964901966,
12.8985548493,
12.9006183889,
12.9026808161,
12.9047421313,
12.9068023347,
12.9088614271,
12.9109194087,
12.9129762799,
12.9150320414,
12.9170866934,
12.9191402365,
12.921192671,
12.9232439974,
12.9252942162,
12.

15.5205473733,
15.5202954573,
15.5200431699,
15.519790511,
15.5195374801,
15.5192840772,
15.5190303021,
15.5187761544,
15.5185216341,
15.5182667408,
15.5180114744,
15.5177558346,
15.5174998212,
15.517243434,
15.5169866728,
15.5167295373,
15.5164720274,
15.5162141429,
15.5159558834,
15.5156972488,
15.5154382389,
15.5151788535,
15.5149190923,
15.5146589551,
15.5143984417,
15.5141375519,
15.5138762854,
15.5136146421,
15.5133526217,
15.5130902241,
15.5128274489,
15.512564296,
15.5123007652,
15.5120368562,
15.5117725689,
15.5115079029,
15.5112428582,
15.5109774344,
15.5107116314,
15.5104454489,
15.5101788867,
15.5099119446,
15.5096446225,
15.50937692,
15.5091088369,
15.5088403731,
15.5085715283,
15.5083023023,
15.5080326948,
15.5077627058,
15.5074923349,
15.5072215819,
15.5069504466,
15.5066789289,
15.5064070284,
15.506134745,
15.5058620784,
15.5055890285,
15.505315595,
15.5050417776,
15.5047675763,
15.5044929907,
15.5042180207,
15.503942666,
15.5036669264,
15.5033908017,
15.5031142917,
15.

6.50627436298,
6.50130409686,
6.49633216049,
6.49135855494,
6.4863832813,
6.48140634063,
6.47642773403,
6.47144746259,
6.4664655274,
6.46148192957,
6.4564966702,
6.4515097504,
6.4465211713,
6.441530934,
6.43653903965,
6.43154548937,
6.4265502843,
6.42155342558,
6.41655491437,
6.41155475181,
6.40655293906,
6.40154947729,
6.39654436767,
6.39153761137,
6.38652920957,
6.38151916345,
6.37650747421,
6.37149414305,
6.36647917116,
6.36146255975,
6.35644431004,
6.35142442324,
6.34640290057,
6.34137974326,
6.33635495255,
6.33132852968,
6.32630047588,
6.32127079242,
6.31623948055,
6.31120654153,
6.30617197662,
6.3011357871,
6.29609797425,
6.29105853936,
6.2860174837,
6.28097480858,
6.27593051529,
6.27088460515,
6.26583707947,
6.26078793956,
6.25573718674,
6.25068482235,
6.24563084773,
6.2405752642,
6.23551807313,
6.23045927586,
6.22539887374,
6.22033686815,
6.21527326045,
6.21020805202,
6.20514124425,
6.20007283851,
6.1950028362,
6.18993123872,
6.18485804747,
6.17978326388,
6.17470688934,
6.16962